In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# rawdata 
#change here
id = '1W--ECnZ7uvksw5b2nEvzFDIO3I0e5_Gw'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('raw.csv')  


In [2]:
import pandas as pd
import numpy as np
import re,string
from lxml import etree
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import Word2Vec
import warnings

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [3]:
data=pd.read_csv('raw.csv')
data.shape

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(149922, 18)

In [4]:
#drop duplicates
data=data.drop_duplicates()
data=data.reset_index()
data.shape

(109072, 19)

In [5]:
data.columns

Index(['index', 'id_str', 'from_user', 'text', 'created_at', 'time',
       'geo_coordinates', 'user_lang', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'from_user_id_str',
       'in_reply_to_status_id_str', 'source', 'profile_image_url',
       'user_followers_count', 'user_friends_count', 'user_location',
       'status_url', 'entities_str'],
      dtype='object')

In [0]:
data['clean_text']=None
data['urls']=None
df=data.loc[:,['id_str','text','clean_text','urls']] 

In [0]:
# try:  
#     highpoints = re.compile(u'[\U00010000-\U0010ffff]')  
# except re.error:    
#     highpoints = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')

for i in range(data.shape[0]):
    try:
      #seperate urls
      urls=''
      url=re.findall(r"(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*,]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)|([a-zA-Z]+.\w+\.+[a-zA-Z0-9\/_]+)",data.text[i])
      if len(url)>0:
          for u in url:
              urls=u[0]+','+urls
      text=data.text[i]
      #remove urls
      text=re.sub(r"(http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*,]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)|([a-zA-Z]+.\w+\.+[a-zA-Z0-9\/_]+)",'',text)
      #remove RT @...:
      text=re.sub(r'(RT @.*?:)','',text)
      #remove @user
      text=re.sub(r'@.*? ','',text)
      text=text.strip()
  #     #remove emoji
  #     text = highpoints.sub(u'??',text)
      
#       #remove punctuation    
#       text=re.sub('[%s]' % re.escape(string.punctuation), '', text)
#       #lowercase
#       text=text.lower()
      #removing non-alphanumeric characters
      text=re.sub(r'[^a-zA-Z0-9\s#\']','',text)
      
      
      #remove \n & \r
      text=text.replace('\n',' ')
      text=text.replace('\r',' ')
      text=text.strip()
      df.clean_text[i]=text
      df.urls[i]=urls.strip(',')
    except:
      df.urls[i]=-1
df=df[df.urls!=-1]
df=df.reset_index()
df=df.drop(['index'],axis=1)

In [0]:
# df_drop=df.drop_duplicates()
# df_drop=df_drop.reset_index()
# df_drop=df_drop.drop(['index'],axis=1)
df_drop=df

In [9]:
! pip install langdetect
import langdetect
from langdetect import detect

    100% |████████████████████████████████| 1.0MB 14.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect


In [0]:
for i in range(df_drop.shape[0]):
    try:
        if detect(df_drop.clean_text[i])!='en':
            df_drop.clean_text[i]=1
    except:
        df_drop.clean_text[i]=1

In [0]:
# df_drop=df_drop[df_drop.clean_text!=1]
# df_drop=df_drop.reset_index()
# df_drop=df_drop.drop(['index'],axis=1)

In [12]:
df_drop

,id_str,text,clean_text,urls
0,1111417608056664066,RT @NealKovach: Sania Nishtar says that we wil...,Sania Nishtar says that we will overcome the s...,https://t.co/ZOnygNaaT2
1,1111417559146881024,Found out that I had type 1 diabetes a few day...,Found out that I had type 1 diabetes a few day...,
2,1111417545997746178,@SongbirdN7 I tried that. All I got was a gut ...,I tried that All I got was a gut and diabetes,
3,1111417532873768960,RT @NHSMillion: This person had to spend $807....,This person had to spend 80796 for the one thi...,https://t.co/R3KYkWCFJN
4,1111417460001890310,If you know anyone in these areas with diabete...,If you know anyone in these areas with diabete...,https://t.co/mjEfzM3hsT
5,1111417442331316230,".@HardballChris: ""How do have protections for ...",How do have protections for people with preexi...,https://t.co/3YvPFstBiz
6,1111417370952663043,RT @bigfatsurprise: USDA official saying that ...,USDA official saying that #DietaryGuidelines w...,https://t.co/cUOqn1ugNZ
7,1111417347275804672,@fakelud Tem glicemia. To com suspeita de diab...,1,
8,1111417326274928640,"RT @IRIStelemed: 400,000!!! We have performed ...",400000 We have performed over 400K #diabeticre...,https://t.co/Y8C1A4kjQC
9,1111417282704474112,"RT @WarriorWifeMom: It isn’t speech delay, it ...",It isnt speech delay it is vaccine damage I...,


In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#change here
df_drop.to_csv('/content/gdrive/My Drive/5703_project/tweets_3-21-3-28.csv',encoding='utf8', index=False)